# How to cite 
@software{Alireza_Soroudi_PyomoOptimization, author = {Alireza Soroudi, Alireza}, license = {MIT}, title = {{PyomoOptimization}}, url = {https://github.com/OptimizationExpert/Pyomo%7D }

In [1]:
from pyomo.environ import *

# Single level optimization problem
$$\min_{x,y} OF = 3x+y $$
$$(-x-4y+8) \leq 0  : \mu_1$$
$$(x+2y-13) \leq 0  : \mu_2$$
$$(x+y-8) \leq 0  : \mu_3$$
$$  1\leq x \leq 6 : \mu_4$$


# Bi-level optimization problem
$$\min_{x} OF_{upper} = 3x+y $$
$$  1\leq x \leq 6 : \mu_4$$
Subject to 
$$\min_{y} OF_{lower} = -y $$
$$(-x-4y+8) \leq 0  : \mu_1$$
$$(x+2y-13) \leq 0  : \mu_2$$
$$(x+y-8) \leq 0  : \mu_3$$


We need to replace the lower problem with its KKT conditions 

# KKT conditions: 
- Create Lagrangian      
$$\mathcal{L} = -y + \mu_1(-x-4y+8) +\mu_2(x+2y-13)+\mu_3(x+y-8)$$

- Differentiate to the lower leb=vel variable (y) = 0       
$$ \frac{\partial \mathcal{L}}{\partial y} = -1 -4\mu_1+2\mu_2+\mu_3 =0  $$

- Complementarity slackness conditions 
$$\mu_1(-x-4y+8)=0 $$
$$\mu_2(x+2y-13)=0$$
$$\mu_3(x+y-8)=0  $$
$$ \mu_i \geq 0 $$

- Original conditions: 
$$(-x-4y+8) \leq 0$$
$$(x+2y-13) \leq 0$$
$$(x+y-8) \leq 0$$
$$  1\leq x \leq 6 : \mu_4$$


# Pyomo code

In [2]:
# create a model
model = AbstractModel()
# declare decision variables
model.x = Var(bounds= (1,6) , domain=Reals)
model.y = Var(domain=Reals)
# declare constraints
model.C1 = Constraint(expr = (-model.x-4*model.y+8) <= 0)
model.C2 = Constraint(expr = (model.x+2*model.y-13) <= 0)
model.C3 = Constraint(expr = (model.x+model.y-8) <= 0)
# declare objective
model.obj = Objective(expr = model.x + 3*model.y, sense=minimize)
opt = SolverFactory('glpk')
instance = model.create_instance()# solves and updates instance
results = opt.solve(instance)
if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
    print('feasible')
elif (results.solver.termination_condition == TerminationCondition.infeasible):
    print('infeasible')
else:
    print ('Solver Status:',  results.solver.status)
print(value(instance.obj))
print('x = ', round(value(instance.x),4) )
print('y = ', round(value(instance.y),4) )

feasible
6.25
x =  1.0
y =  1.75


In [3]:
model = AbstractModel()
# declare decision variables
model.x = Var(bounds= (1,6) , domain=Reals)
model.y = Var(domain=Reals)
model.mu1 = Var(bounds= (0,None) , domain=Reals)
model.mu2 = Var(bounds= (0,None) , domain=Reals)
model.mu3 = Var(bounds= (0,None) , domain=Reals)

# declare constraints
model.C1 = Constraint(expr = (-model.x-4*model.y+8) <= 0)
model.C2 = Constraint(expr = (model.x+2*model.y-13) <= 0)
model.C3 = Constraint(expr = (model.x+model.y-8) <= 0)

model.D1 = Constraint(expr = model.mu1*(-model.x-4*model.y+8) == 0)
model.D2 = Constraint(expr = model.mu2*(model.x+2*model.y-13) == 0)
model.D3 = Constraint(expr = model.mu3*(model.x+model.y-8) == 0)

model.lagrngian = Constraint(expr = -1 -4*model.mu1+2*model.mu2+model.mu3 == 0)

# declare objective
model.obj = Objective(expr = model.x + 3*model.y, sense=minimize)
opt = SolverFactory('ipopt')
instance = model.create_instance()# solves and updates instance
results = opt.solve(instance)
if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
    print('feasible')
elif (results.solver.termination_condition == TerminationCondition.infeasible):
    print('infeasible')
else:
    print ('Solver Status:',  results.solver.status)
print('OF = ', round(value(instance.obj),4) )
print('x = ', round(value(instance.x),4) )
print('y = ', round(value(instance.y),4) )
print('mu1 = ', round(value(instance.mu1),4) )
print('mu2 = ', round(value(instance.mu2),4) )
print('mu3 = ', round(value(instance.mu3),4) )

feasible
OF =  12.0
x =  6.0
y =  2.0
mu1 =  0.0
mu2 =  0.0
mu3 =  1.0
